# Initialisation

In [1]:
import huggingface_hub
from huggingface_hub import snapshot_download
import os
import pandas as pd
import gc
import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
"""import torch
import gc
torch.cuda.empty_cache()
gc.collect()

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))"""
!nvidia-smi

#!pkill -u $USER -f python

In [ ]:
"""!kill 2086774 

del tokenizer
del model
gc.collect()
torch.cuda.empty_cache()"""

In [ ]:
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))

import bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

model_name = "unsloth/Mistral-Small-Instruct-2409-bnb-4bit"
#model_name = "unsloth/Phi-4-mini-instruct-bnb-4bit"
#model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"
#model_name = "unsloth/Qwen2-7B-Instruct-bnb-4bit"
#model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

config = AutoConfig.from_pretrained(model_name)
config.attn_implementation = "eager"
config.output_attentions = True
config.output_hidden_states = False 

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    device_map="auto",
)

model.eval()
model.to("cuda:0")

print(model.num_parameters())

In [ ]:
!nvidia-smi

# Gender

In [ ]:
import torch
import re

promptSize = 849  # longueur fixe pour découper le résultat
max_prompt_tokens = 300  # limiter le nombre de tokens des lyrics pour économiser de la mémoire

def prompt_gender(lyrics, tokenizer=None, max_prompt_tokens=None, device="cuda:0"):
    # Tronquer uniquement les lyrics si max_prompt_tokens défini
    if max_prompt_tokens is not None and tokenizer is not None:
        tokenized_lyrics = tokenizer(lyrics, return_tensors="pt").input_ids[0]
        tokenized_lyrics = tokenized_lyrics[:max_prompt_tokens].to(device)
        lyrics = tokenizer.decode(tokenized_lyrics, skip_special_tokens=True)

    return f"""You are a gender classifier that labels song lyrics based on whether the narrator appears to be male, female, or neutral. Use lyrical content, tone, and perspective to decide. Your answer must include specific words or phrases from the lyrics that influenced your decision. Return the result using this format:

LYRICS: <lyrics>  
GENDER: <male|female|neutral>  
KEYWORDS: <list of specific words or expressions from the lyrics>

EXAMPLES:

LYRICS: I wear my heart upon my sleeve, like a girl who's never been hurt before  
GENDER: female  
KEYWORDS: "girl", "wear my heart upon my sleeve"

LYRICS: Got my truck and my beer, ain't got no time for games  
GENDER: male  
KEYWORDS: "truck", "beer", "ain't got no time"

LYRICS: The sky is open, my soul is light, I drift where the wind tells me to  
GENDER: neutral  
KEYWORDS: "sky", "soul", "wind"

LYRICS: {lyrics}  
GENDER:"""

def getGenre(result):
    result = result[promptSize:]
    match = re.search(r"GENDER:\s*(male|female|neutral)", result, re.IGNORECASE)
    return match.group(1) if match else None

def getGenreLLM_with_attention_and_hidden(lyrics, model, tokenizer, device="cuda:0"):
    prompt = prompt_gender(lyrics, tokenizer=tokenizer, max_prompt_tokens=max_prompt_tokens, device=device)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    model.config.attn_implementation = "eager"
    model.config.output_attentions = True
    model.config.output_hidden_states = True

    model.eval()
    with torch.no_grad():
        # Forward sur le prompt initial
        outputs = model(**inputs, output_attentions=True, output_hidden_states=True)

        attentions_prompt = [att.clone().detach().cpu() for att in outputs.attentions]
        hidden_states_prompt_full = outputs.hidden_states[-1].clone().detach().cpu()

        # Générer un seul token supplémentaire (greedy top-1)
        last_token_logits = outputs.logits[:, -1, :]
        next_token_id = torch.argmax(last_token_logits, dim=-1).unsqueeze(-1)
        input_with_next = torch.cat([inputs["input_ids"], next_token_id], dim=-1)

        # Forward pour le prompt + token généré
        outputs_next = model(input_ids=input_with_next, output_attentions=True, output_hidden_states=True)

        attentions_next = [att.clone().detach().cpu() for att in outputs_next.attentions]
        hidden_states_next_full = outputs_next.hidden_states[-1].clone().detach().cpu()

        # Extraire uniquement la partie correspondant aux derniers lyrics et au token généré
        #prompt_without_lyrics = tokenizer(prompt[0:promptSize], return_tensors="pt").input_ids.to(device)
        #len_prompt = prompt_without_lyrics.shape[1]+ 1
        
        len_prompt = 260
        hidden_states_prompt = hidden_states_prompt_full[:, len_prompt:, :]
        hidden_states_next = hidden_states_next_full[:, len_prompt:, :]

        # Décodage pour récupérer genre
        generated_text = tokenizer.decode(input_with_next[0], skip_special_tokens=True)
        gender = getGenre(generated_text)

        #del prompt_without_lyrics
        del outputs, outputs_next, inputs, input_with_next, next_token_id, hidden_states_prompt_full, hidden_states_next_full
        torch.cuda.empty_cache()
        gc.collect()

    return gender, attentions_prompt, attentions_next, hidden_states_prompt, hidden_states_next, generated_text



# --- Exemple d'utilisation ---
torch.cuda.empty_cache()
lyrics = """NA Yeah, Spyderman and Freeze in full effect Uh-huh You ready, Ron? I'm ready You ready, Biv? I'm ready, Slick, are you? Oh, yeah, break it down NA Girl, I, must (warn you) I sense something strange in my mind Situation is (serious) Let's cure it cause we're running out of time It's oh, so (beautiful) Relationships they seem from the start It's all so (deadly) When love is not together from the heart It's drivin' me out of my mind! That's why it's HARD for me to find Can't get it out of my head! Miss her, kiss her, love her(Wrong move you're dead!) That girl is (poison)...Never trust a big butt and smile That girl is (poison)..("POISON!!") NA (-caution) Before I start to meet a fly girl, you know? Cause in some (portions) You'll think she's the best thing in the world She's so - (fly) She'll drive you right out of your mind And steal your heart when you're blind Beware she's schemin', she'll make you think you're dreamin' YOU'LL fall in love and you'll be screamin', demon, HOO.. Poison, deadly, movin' in slow Lookin for a mellow fellow like DeVoe Gettin paid, laid, so better lay low Schemin on house, money, and the whole show The low pro ho she'll be cut like an aaa-FRO See what you're sayin', huh, she's a winner to you But I know she's a loser (How do you know?) Me and the crew used to do her! "POISON!" "POISON!" "POISON!" "POISON!" "POISON!" "POISON!" "POISON!" "POISON! "POISON!" "POISON!" "POISON!" "POISON! "POISON!" "POISON!" "POISON!" "POISON! I was at the park, shake, breakin and takin 'em all And that night, I played the wall Checkin' out the fellas, the highs and lows Keepin' one eye open, still clockin' the hoes There was one particular girl that stood out from the rest Poison as can be, the high power chest Michael Biv here and I'm runnin' the show Bell, Biv DeVoe ..now you know! Yo, Slick, blow.. It's drivin' me out of my mind! That's why it's HARD for me to find Can't get it out of my head! Miss her, kiss her, love her(Wrong move you're dead!) That girl is (poison)...Never trust a big butt and smile That girl is (poison)..("POISON!!") Yo' fellas, that was my end of.. You know what I'm sayin', Mike? Yeah, B.B.D. in full effect Yo', wassup to Ralph T and Johnny G And I can't forget about my boy, B. Brown And the whole NE crew Poison.. NA"""
gender, attentions_prompt, attentions_next, hidden_states_prompt, hidden_states_next, generated_text = getGenreLLM_with_attention_and_hidden(lyrics, model, tokenizer)

print(f"Genre : {gender}")
print(f"Generated text: {generated_text}")
print("Shape attentions couche dernière (prompt) :", attentions_prompt)
print("Shape attentions couche dernière (prompt + token) :", attentions_next)
print("Shape hidden_states dernière couche (lyrics) :", hidden_states_prompt.shape)
print("Shape hidden_states dernière couche (lyrics + token) :", hidden_states_next.shape)


In [ ]:
# Définir le nom du modèle et le chemin de sortie

modele_str = model_name.split('/')[-1]
PATH_output_gender = f'/home/evuichard/Projet DEBIAR/labeled_lyrics_{modele_str}_attention&hidden_states.xlsx'

# Vérifier si le fichier de sortie existe et s'il est correctement formaté
def need_init(path):
  if not os.path.isfile(path):
    return True
  df_check = pd.read_excel(path, sheet_name="Sheet1")
  required_cols = [
    'genre_LLM',
    'attentions_prompt',
    'attentions_next',
    'hidden_states_prompt',
    'hidden_states_next'
  ]
  for col in required_cols:
    if col not in df_check.columns:
      return True
  if df_check['genre_LLM'].notna().sum() <= 200:
    return True
  return False

if False and need_init(PATH_output_gender):
  df = pd.read_excel('/home/evuichard/Projet DEBIAR/translated_lyrics.xlsx', sheet_name="Sheet1")
  df['genre_LLM'] = None
  df['attentions_prompt'] = None
  df['attentions_next'] = None
  df['hidden_states_prompt'] = None
  df['hidden_states_next'] = None
  df.to_excel(PATH_output_gender, index=False)

df = pd.ExcelFile(PATH_output_gender)
df = df.parse("Sheet1")
df['genre_LLM'] = df['genre_LLM'].astype('object')

In [ ]:
from IPython.display import clear_output
import os
import torch

# Créer le dossier de sortie s'il n'existe pas
tensor_output_dir = f"/home/evuichard/Projet DEBIAR/labeled_lyrics_tensors_{modele_str}/gender/"
os.makedirs(tensor_output_dir, exist_ok=True)

for index, row in df.iterrows():
    if index % 100 == 0:
        clear_output(wait=True)
        print(f"Processing index: {index} / {len(df)}")

    # On skip si genre déjà calculé
    if len(str(df['genre_LLM'][index])) > 3:
        print(index, end=' ')
        continue

    lyrics = df['english_lyrics'][index]

    # Calculer genre + attentions + hidden states
    gender, attentions_prompt, attentions_next, hidden_states_prompt, hidden_states_next, generated_text = getGenreLLM_with_attention_and_hidden(
        lyrics, model, tokenizer
    )

    # Mettre à jour le DataFrame pour le suivi
    df.loc[index, 'genre_LLM'] = gender

    # Préparer le dict à sauvegarder
    data_to_save = {
        'genre': gender,
        'generated_text': generated_text,
        'attentions_prompt': attentions_prompt,
        'attentions_next': attentions_next,
        'hidden_states_prompt': hidden_states_prompt,
        'hidden_states_next': hidden_states_next
    }

    # Sauvegarder un fichier .pt par ligne
    file_path = os.path.join(tensor_output_dir, f"line_{index}.pt")
    torch.save(data_to_save, file_path)

    print(f"{index} saved, genre: {gender}")

    # Nettoyage mémoire
    del attentions_prompt, attentions_next, hidden_states_prompt, hidden_states_next, data_to_save
    torch.cuda.empty_cache()


In [ ]:
#mettre en minuscule tous les labels de genre
df['genre_LLM'] = df['genre_LLM'].str.lower()
df.to_excel(PATH_output_gender)

In [ ]:
from termcolor import colored

def weight_to_ansi(weight):
    if weight < 0.5:
        ratio = weight / 0.5
        r = int(200 * (1 - ratio) + 200 * ratio)
        g = int(200 * (1 - ratio) + 0 * ratio)
        b = int(200 * (1 - ratio) + 200 * ratio)
    else:
        ratio = (weight - 0.5) / 0.5
        r = int(200 * (1 - ratio) + 255 * ratio)
        g = 0
        b = int(200 * (1 - ratio) + 0 * ratio)
        
    ansi_code = rgb_to_ansi(r, g, b)
    return ansi_code

def rgb_to_ansi(r, g, b):
    """
    Convertit une couleur RGB en code ANSI 256.
    """
    # On approxime à 6 niveaux (0-5)
    r = int(r / 51)
    g = int(g / 51)
    b = int(b / 51)
    return 16 + 36*r + 6*g + b

In [ ]:
def index_token_to_word(index, tokenizer, tokens = [], prompt =""):
    if not tokens:
        tokens = tokenizer.encode(prompt)
    if abs(index) >= len(tokens):
        return None
    print(tokens)
    return tokenizer.decode([tokens[index]])

def index_word_to_token(word, tokenizer, tokens = [], prompt =""):
    if not tokens:
        tokens = tokenizer.encode(prompt)
    token = tokenizer.encode(word)
    if token not in tokens:
        return None
    return tokens.index(token)

def visualize_token_indices(prompt, tokenizer, line_width=100):
    tokens = tokenizer.encode(prompt)
    words = [tokenizer.decode([t]) for t in tokens]

    current_words, current_indices = "", ""
    char_count = 0

    for i, w in enumerate(words):
        # si le token contient un saut de ligne -> on affiche le bloc avant de repartir
        if "\n" in w:
            word_display = colored(w.replace("\n", ""), 'green', attrs=['bold'])
            current_words += word_display + " "
            
            idx_str = str(i)
            padding = max(len(w.replace("\n", "")) - len(idx_str), 0)
            current_indices += " " * padding + colored(idx_str, 'cyan', attrs=['bold']) + " "
            
            # impression du bloc courant
            print(current_words.rstrip())
            print(current_indices.rstrip())
            print()  # ligne vide pour séparer

            # réinitialiser
            current_words, current_indices = "", ""
            char_count = 0
            continue

        # ajout normal
        word_display = colored(w, 'green', attrs=['bold'])
        current_words += word_display + " "
        idx_str = str(i)
        padding = max(len(w) - len(idx_str), 0)
        current_indices += " " * padding + colored(idx_str, 'cyan', attrs=['bold']) + " "

        char_count += len(w) + 1
        if char_count > line_width:
            print(current_words.rstrip())
            print(current_indices.rstrip())
            print()
            current_words, current_indices = "", ""
            char_count = 0

    # afficher le dernier bloc si non vide
    if current_words.strip():
        print(current_words.rstrip())
        print(current_indices.rstrip())



visualize_token_indices(prompt_gender(""), tokenizer)

def visualize_weight_sentence(tokenizer, tokens, weights):
    """
    Affiche une phrase avec les tokens colorés suivant leur poids d'importance.
    Dégradé continu du gris clair au rouge vif.
    """
    print(len(tokens), len(weights))
    if not isinstance(weights, torch.Tensor):
        weights = torch.tensor(weights)

    # Normalisation des poids
    weights = (weights - weights.mean()) / (weights.std() + 1e-5)  # z-score
    weights = (weights - weights.min()) / (weights.max() - weights.min() + 1e-5)

    spectrum_steps = 10
    print("Spectre d'importance :")
    for i in range(spectrum_steps):
        weight = i / (spectrum_steps - 1)
        ansi_code = weight_to_ansi(weight)
        print(f"\033[48;5;{ansi_code}m \033[0m", end='')
    print("\n") 
    
    # Décoder les tokens
    decoded_tokens = tokenizer.convert_ids_to_tokens(tokens)
    
    for token, weight in zip(decoded_tokens, weights):
        # Couleur : du gris clair (200,200,200) au rouge vif (255,0,0)
        ansi_code = weight_to_ansi(weight)
        print(f"\033[38;5;{ansi_code}m{token}\033[0m", end='')
    print()

tokens = tokenizer.encode(prompt_gender("")[:50])
visualize_weight_sentence(tokenizer, tokens, np.random.rand(len(tokens)))

In [ ]:
import json
import numpy as np
from IPython.core.display import display, HTML

def call_html():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

def show_attention_html(tokenizer, attention, tokens, max_tokens_display=300):
    """
    Affiche une view HTML interactive de la matrice d'attention.
    Params:
      tokenizer: tokenizer avec méthode convert_ids_to_tokens(list_of_ids)
      attention: np.array or torch tensor shape (L, H, qlen, klen)
      tokens: list/array of token ids (length should match qlen and klen if self-attention)
      max_tokens_display: safety cap to avoid insanely long exports
    """
    # Convert to numpy
    if hasattr(attention, 'detach'):  # torch tensor
        attention = attention.detach().cpu().numpy()
    else:
        attention = np.array(attention)

    if attention.ndim != 4:
        raise ValueError("attention must be shape (layers, heads, qlen, klen)")

    layers, heads, qlen, klen = attention.shape

    # optionally truncate tokens for display if too long
    if len(tokens) > max_tokens_display:
        raise ValueError(f"tokens length {len(tokens)} > max_tokens_display {max_tokens_display}. Truncate before calling.")

    # decode tokens to readable strings (subword tokens OK)
    try:
        decoded = tokenizer.convert_ids_to_tokens(list(tokens))
    except Exception:
        # fallback: try tokenizer.decode on each id
        decoded = []
        for t in tokens:
            try:
                decoded.append(tokenizer.decode([int(t)]))
            except Exception:
                decoded.append(str(t))

    # normalize attention per-layer/head for nicer visualization (we keep raw values but normalize to 0..1)
    # We'll produce a nested python list: att_list[layer][head] = 2D list qlen x klen
    att_list = []
    for l in range(layers):
        layer_list = []
        for h in range(heads):
            mat = attention[l, h]
            # clip NaN/inf
            mat = np.nan_to_num(mat, nan=0.0, posinf=0.0, neginf=0.0)
            # normalize per-matrix to 0..1 for coloring/stroke width (keeps relative shape)
            mmin, mmax = mat.min(), mat.max()
            if mmax - mmin < 1e-8:
                norm = np.zeros_like(mat).tolist()
            else:
                norm = ((mat - mmin) / (mmax - mmin)).tolist()
            layer_list.append(norm)
        att_list.append(layer_list)

    data = {
        "tokens": decoded,
        "layers": att_list  # shape L x H x q x k
    }

    # create HTML + JS (D3 v3) viewer
    # color mapping function in JS will map weight [0..1] -> rgb (grey->violet->red)
    html = f"""
    <div id="att_viewer_{id(attention)}" style="font-family: Arial, Helvetica, sans-serif;"></div>
    <script>
    (function() {{
      require(['d3','jquery'], function(d3, $) {{
        var data = {json.dumps(data)};
        var tokens = data.tokens;
        var layers = data.layers;
        var L = layers.length;
        var H = layers[0].length;
        var container = d3.select("#att_viewer_{id(attention)}");
        container.html(''); // reset

        // Controls
        var controlDiv = container.append('div').style('margin-bottom','6px');
        controlDiv.append('label').text('Layer: ');
        var layerSel = controlDiv.append('select').attr('id','layer_select');
        for (var i=0;i<L;i++) {{
          layerSel.append('option').attr('value',i).text(i);
        }}
        controlDiv.append('span').style('margin','0 10px');
        controlDiv.append('label').text('Head: ');
        var headSel = controlDiv.append('select').attr('id','head_select');
        headSel.append('option').attr('value','all').text('All');
        for (var h=0; h<H; h++) {{
          headSel.append('option').attr('value',h).text(h);
        }}

        // Spectrum legend (grey -> violet -> red)
        var legend = container.append('div').style('margin','6px 0');
        legend.append('span').text('Spectrum: ');
        var spec = legend.append('span').style('display','inline-block').style('vertical-align','middle');
        var steps = 40;
        for (var s=0;s<steps;s++){{
          var w = s/(steps-1);
          // color computed same as viewer mapping
          var col = weightToRgbCss(w);
          spec.append('span').style('display','inline-block')
                        .style('width','10px').style('height','10px').style('background',col);
        }}

        // SVG area
        var width = Math.min(1000, Math.max(600, tokens.length*18));
        var height = Math.min(900, Math.max(300, tokens.length*12));
        var svg = container.append('svg').attr('width', width).attr('height', height);
        svg.style('background', '#1f1f1f');

        // layout constants
        var leftX = 120, rightX = width - 120;
        var marginTop = 20;
        var spacing = (height - 2*marginTop) / Math.max(tokens.length,1);
        var tokenY = function(i) {{ return marginTop + i*spacing + spacing/2; }};

        // draw token labels
        var leftGroup = svg.append('g').attr('class','left');
        var rightGroup = svg.append('g').attr('class','right');

        leftGroup.selectAll('text')
          .data(tokens)
          .enter()
          .append('text')
          .attr('x', leftX - 10)
          .attr('y', function(d,i) {{ return tokenY(i); }})
          .attr('text-anchor','end')
          .style('fill','#dcdcdc')
          .style('font-family','monospace')
          .style('font-size','11px')
          .text(function(d){{
            // short token display
            return d.length>20 ? d.slice(0,18)+'..' : d;
          }});

        rightGroup.selectAll('text')
          .data(tokens)
          .enter()
          .append('text')
          .attr('x', rightX + 10)
          .attr('y', function(d,i) {{ return tokenY(i); }})
          .attr('text-anchor','start')
          .style('fill','#dcdcdc')
          .style('font-family','monospace')
          .style('font-size','11px')
          .text(function(d){{
            return d.length>20 ? d.slice(0,18)+'..' : d;
          }});

        // container for links
        var linkGroup = svg.append('g').attr('class','links');

        // helper: color mapping grey->violet->red for weight in [0,1]
        function weightToRgbCss(w) {{
          // w in [0,1]
          var r,g,b;
          if (w < 0.5) {{
            var ratio = w / 0.5;
            r = Math.round(200*(1-ratio) + 200*ratio);
            g = Math.round(200*(1-ratio) + 0*ratio);
            b = Math.round(200*(1-ratio) + 200*ratio);
          }} else {{
            var ratio = (w-0.5)/0.5;
            r = Math.round(200*(1-ratio) + 255*ratio);
            g = 0;
            b = Math.round(200*(1-ratio) + 0*ratio);
          }}
          return 'rgb('+r+','+g+','+b+')';
        }}

        // draw/update function
        function update() {{
          var layer = +d3.select('#layer_select').node().value;
          var headVal = d3.select('#head_select').node().value;
          var mat; // q x k
          if (headVal === 'all') {{
            // average across heads
            var sum = [];
            var first = layers[layer][0];
            var q = first.length, k = first[0].length;
            for (var i=0;i<q;i++) {{
              sum.push(new Array(k).fill(0.0));
            }}
            for (var h=0; h<layers[layer].length; h++) {{
              var m = layers[layer][h];
              for (var i=0;i<q;i++) for (var j=0;j<k;j++) sum[i][j] += m[i][j];
            }}
            for (var i=0;i<q;i++) for (var j=0;j<k;j++) sum[i][j] /= layers[layer].length;
            mat = sum;
          }} else {{
            mat = layers[layer][+headVal];
          }}

          // flatten matrix entries into list of links with positions and weight
          var links = [];
          var q = mat.length, k = mat[0].length;
          for (var i=0;i<q;i++) {{
            for (var j=0;j<k;j++) {{
              var w = mat[i][j];
              // optional threshold: don't show very tiny links to reduce clutter
              if (w <= 0.01) continue;
              links.push({{source:i, target:j, weight:w}});
            }}
          }}

          // bind data
          var selection = linkGroup.selectAll('path.link').data(links, function(d) {{ return d.source+','+d.target; }});

          // exit
          selection.exit().transition().duration(200).style('opacity',0).remove();

          // enter
          var enter = selection.enter().append('path').attr('class','link')
            .attr('fill','none')
            .attr('stroke-linecap','round')
            .attr('stroke-opacity',0.9)
            .attr('stroke-width',1)
            .style('mix-blend-mode','screen');

          // update + enter merged
          selection = enter.merge(selection);
          selection.each(function(d) {{
            var sy = tokenY(d.source);
            var ty = tokenY(d.target);
            // nice curved path between leftX and rightX
            var mx = (leftX + rightX)/2;
            var path = 'M'+leftX+','+sy+' C'+mx+','+sy+' '+mx+','+ty+' '+rightX+','+ty;
            d3.select(this).attr('d', path)
              .attr('stroke', weightToRgbCss(d.weight))
              .attr('stroke-width', Math.max(0.4, 6 * d.weight))  // stroke width scaling (tweakable)
              .attr('opacity', Math.min(1.0, 0.3 + 0.7*d.weight));
          }});
        }}

        // first draw
        update();

        // events
        d3.select('#layer_select').on('change', update);
        d3.select('#head_select').on('change', update);

      }}); // require
    }})(); 
    </script>
    """

    # display loader + viewer
    call_html()
    display(HTML(html))

# Example usage:
# show_attention_html(tokenizer, attention_np_array, tokens_list)


In [ ]:
from IPython.core.display import display, HTML


def visualize_attention_html(tokenizer, attention, tokens):
    """
    Affiche une matrice d'attention en HTML.
    
    tokenizer : tokenizer pour décoder les tokens
    attention : np.array ou torch.Tensor de shape (num_layers, num_heads, seq_len, seq_len)
    tokens : tensor ou liste des tokens d'entrée
    
    layer : couche de la matrice à visualiser
    head : tête à visualiser
    """
    # Si torch tensor, convertir en numpy
    if 'torch' in str(type(attention)):
        attention = attention.detach().cpu().numpy()
    
    # Sélection de la couche et tête
    att_matrix = attention
    
    # Décoder les tokens
    decoded_tokens = tokenizer.convert_ids_to_tokens(tokens)
    
    # Normalisation pour couleur
    att_norm = (att_matrix - att_matrix.min()) / (att_matrix.max() - att_matrix.min() + 1e-8)
    
    # Générer tableau HTML
    html = "<table style='border-collapse: collapse;'>"
    
    # Première ligne : tokens en colonnes
    html += "<tr><th></th>"
    for tok in decoded_tokens:
        html += f"<th style='padding:2px; font-family:monospace;'>{tok}</th>"
    html += "</tr>"
    
    # Lignes de la matrice
    for i, tok_row in enumerate(decoded_tokens):
        html += f"<tr><th style='padding:2px; font-family:monospace;'>{tok_row}</th>"
        for j in range(len(decoded_tokens)):
            val = att_norm[i, j]
            # Dégradé bleu clair → bleu foncé
            r = int(255*(1-val))
            g = int(255*(1-val))
            b = int(255*val)
            color = f'rgb({r},{g},{b})'
            html += f"<td style='background-color:{color}; width:20px; height:20px; text-align:center;'>{val:.2f}</td>"
        html += "</tr>"
    
    html += "</table>"
    
    display(HTML(html))


In [ ]:
import torch
from sklearn.decomposition import PCA

def PCA_on_dim_n(tensor, dim=0, n_espace_propre=0):
    if tensor.dim() <= dim:
        raise ValueError("La dimension spécifiée est supérieure à la dimension du tenseur.")
    permute_order = list(range(tensor.dim()))
    permute_order.append(permute_order.pop(dim))  
    tensor_perm = tensor.permute(*permute_order)  # shape: (..., target_dim)
    shape_perm = tensor_perm.shape
    tensor_reshaped = tensor_perm.reshape(-1, shape_perm[-1]).detach().cpu().numpy()
    
    pca = PCA()
    pca.fit(tensor_reshaped)
    
    if n_espace_propre >= pca.components_.shape[0]:
        raise ValueError(f"n_espace_propre={n_espace_propre} dépasse le nombre de composantes ({pca.components_.shape[0]})")

    reduced = np.dot(tensor_reshaped, pca.components_[n_espace_propre].T).reshape(*shape_perm[:-1], 1)
    reduced_torch = torch.from_numpy(reduced).to(tensor.device)
    
    inv_permute = [0] * len(permute_order)
    for i, j in enumerate(permute_order):
        inv_permute[j] = i
    reduced_torch = reduced_torch.permute(*inv_permute)

    return reduced_torch.squeeze(dim)



#### Weight - methods ####
## vertical
def weight_vertical_idToken(attention, idToken):
    # attention : (seq_len, seq_len)
    # output : tenseur de dimension (seq_len)
    return attention[:, idToken]

def weight_vertical_token(attention, idToken, tokens):
    indices = [i for i, token in enumerate(tokens) if token == tokens[idToken]]
    return attention[:, indices].mean(dim=1)

def weight_vertical_mean(attention):
    # attention : (seq_len, seq_len)
    # output : tenseur de dimension (seq_len)
    return attention.mean(dim=1)

def weight_vertical_PCA(attention):
    # attention : (seq_len,seq_len)
    # Utilisation de sklearn pour faire une ACP sur l'axe 1 du tenseur
    return PCA_on_dim_n(attention, dim=1)

## horizontal
def weight_horizontal_idToken(attention, idToken):
    # attention : (seq_len, seq_len)
    # output : tenseur de dimension (seq_len)
    return attention[idToken, :]

def weight_horizontal_token(attention, idToken, tokens):
    indices = [i for i, token in enumerate(tokens) if token == tokens[idToken]]
    return attention[indices, :].mean(dim=0)

def weight_horizontal_mean(attention):
    # attention : (seq_len, seq_len)
    # output : tenseur de dimension (seq_len)
    return attention.mean(dim=0)

def weight_horizontal_PCA(attention):
    # attention : (seq_len, seq_len)
    # Chaque colonne est un échantillon, chaque ligne une feature
    return PCA_on_dim_n(attention, dim=0)

In [ ]:
def normalize_tensor(tensor):
    norm = torch.linalg.norm(tensor)
    return tensor / norm

def normalize_tensor_col(tensor):
    # tensor contenant les normes de chaque colonne de la matrice tensor
    norm = torch.linalg.norm(tensor, dim=0, keepdim=True)
    # on divise chaque colonne par sa norme
    return tensor / norm

def normalize_tensor_row(tensor):
    norm = torch.linalg.norm(tensor, dim=1, keepdim=True)
    # on divise chaque ligne par sa norme
    return tensor / norm


In [ ]:
def filtre_id_tensor(tensor,dim,id_start,id_end=0):
    #sur la dimension spécifiée, ne garde que les indices entre id_start et id_end et met le reste à 0 (donc sans réduction de dimension)
    if id_end == 0:
        id_end = tensor.size(dim)
    indices = torch.arange(tensor.size(dim), device=tensor.device)
    mask_dim = (indices >= id_start) & (indices <= id_end)
    shape = [1] * tensor.dim()
    shape[dim] = -1
    mask = mask_dim.view(shape)
    return tensor * mask

def reduce_id_tensor(tensor,dim,id_start,id_end=0):
    # sur la dimension spécifiée, ne garde que les indices entre id_start et id_end, et réduit la dimension
    if id_end == 0:
        id_end = tensor.size(dim)
    return tensor.narrow(dim, id_start, id_end - id_start)

def idee_verticale(attentions, tokens, id_fin_prompt):
    idToken = -1
    indices = [i for i, token in enumerate(tokens) if token == tokens[idToken]]
    filtre_indices = attentions
    print("Shape of filtre_indices:", filtre_indices.shape)
    projection1 = reduce_id_tensor(filtre_indices, dim=-2, id_start=id_fin_prompt)
    projection1 = PCA_on_dim_n(projection1, dim=-1)
    print("Shape of projection1:", projection1.shape)
    projection2 = PCA_on_dim_n(projection1, dim=1)
    print("Shape of projection2:", projection2.shape)
    projection3 = PCA_on_dim_n(projection2, dim=0)
    print("Shape of projection3:", projection3.shape)
    return projection3

def idee_horizontale(attentions, tokens, id_fin_prompt):
    idToken = -1
    indices = [i for i, token in enumerate(tokens) if token == tokens[idToken]]
    filtre_indices = attentions
    print("Shape of filtre_indices:", filtre_indices.shape)
    projection1 = reduce_id_tensor(filtre_indices, dim=-1, id_start=id_fin_prompt)
    projection1 = PCA_on_dim_n(projection1, dim=-2)
    print("Shape of projection1:", projection1.shape)
    projection2 = PCA_on_dim_n(projection1, dim=1)
    print("Shape of projection2:", projection2.shape)
    projection3 = PCA_on_dim_n(projection2, dim=0)
    print("Shape of projection3:", projection3.shape)
    return projection3

In [ ]:
def get_lyrics(df,index):
    return df.loc[index, 'english_lyrics']

def get_prompt_for_line(df,index, troncage = -1, tokenizer = None):
    if troncage > 0:
        lyrics = get_lyrics(df,index)
        tokens = tokenizer.encode(lyrics)
        tokens = tokens[:troncage]
        lyrics = tokenizer.decode(tokens, skip_special_tokens=True)
        return prompt_gender(lyrics)
    return prompt_gender(get_lyrics(df,index))

def load_attention_and_prompt_from_index(index, next = False):
    Path_tensor_gender = f"/home/evuichard/Projet DEBIAR/labeled_lyrics_tensors_{modele_str}/gender/" + f"/line_{index}.pt"
    data_tensor = torch.load(Path_tensor_gender)
    if next:
        attention = torch.stack(data_tensor['attentions_next'])[:,0,:,:,:]
        print("Shape of attention (next):", attention.shape)
        return attention, data_tensor['generated_text']
    attention = torch.stack(data_tensor['attentions_prompt'])[:,0,:,:,:]
    prompt = data_tensor['generated_text']
    c = prompt[-1]
    prompt = prompt[:-1]
    while prompt and c != ' ' and c != ':':
        c = prompt[-1]
        prompt = prompt[:-1]
    return attention, prompt

def attentionsToAttention(attentions):
    #reduction layer
    #attention = attentions.mean(dim=0)
    #attention = attentions[-1]
    attention = PCA_on_dim_n(attentions)
    print("Shape of attention (layer):", attention.shape)
    #reduction head
    #attention = attention.mean(dim=0)
    #attention = attention[-1]
    attention = PCA_on_dim_n(attention)
    print("Shape of attention (head):", attention.shape)
    return attention

def visualize_attention_from_index_and_idToken(index, tokenizer):
    attentions, prompt = load_attention_and_prompt_from_index(index, next=True)
    attention = attentionsToAttention(attentions)
    print("Shape of attention:", attention.shape)

    tokens = tokenizer.encode(prompt)
    print("Shape of tokens:", len(tokens))

    weights = weight_horizontal_token(attention,-1,tokens)
    print("Shape of weights (horizontal, idToken):", weights.shape)
    visualize_weight_sentence(tokenizer, tokens, weights)

    weights = weight_vertical_token(attention,-1,tokens)
    print("Shape of weights (vertical, idToken):", weights.shape)
    visualize_weight_sentence(tokenizer, tokens, weights)

    call_html()
    show_attention_html(tokenizer, attentions[:,:,263:,263:], tokens[263:],306)



df = pd.read_excel('/home/evuichard/Projet DEBIAR/translated_lyrics.xlsx', sheet_name="Sheet1")

visualize_attention_from_index_and_idToken(108, tokenizer)

def visualize_attention_idee(index,tokenizer):
    attentions, prompt = load_attention_and_prompt_from_index(index, next=True)
    tokens = tokenizer.encode(prompt)
    len_prompt = 263 #len(tokenizer.encode(prompt_gender("")))

    weight_horizontal = idee_horizontale(attentions, tokens, len_prompt)
    visualize_weight_sentence(tokenizer, tokens[len_prompt:], weight_horizontal)

    weight_vertical = idee_verticale(attentions, tokens, len_prompt)
    visualize_weight_sentence(tokenizer, tokens[len_prompt:], weight_vertical)

#visualize_attention_idee(90, tokenizer)

# Ethnicity

In [ ]:
print(len("""You are a gender classifier that labels song lyrics based on whether the narrator appears to be male, female, or neutral. Use lyrical content, tone, and perspective to decide. Your answer must include specific words or phrases from the lyrics that influenced your decision. Return the result using this format:

LYRICS: <lyrics>  
GENDER: <male|female|neutral>  
KEYWORDS: <list of specific words or expressions from the lyrics>

EXAMPLES:

LYRICS: I wear my heart upon my sleeve, like a girl who's never been hurt before  
GENDER: female  
KEYWORDS: "girl", "wear my heart upon my sleeve"

LYRICS: Got my truck and my beer, ain't got no time for games  
GENDER: male  
KEYWORDS: "truck", "beer", "ain't got no time"

LYRICS: The sky is open, my soul is light, I drift where the wind tells me to  
GENDER: neutral  
KEYWORDS: "sky", "soul", "wind"

"""))


In [ ]:
promptSize = 1042 #502
def prompt_ethnicity(lyrics):
  return f"""You are an ethnicity classifier. Based on a block of song lyrics written in English, determine the most likely cultural or ethnic background of the narrator. Use references to places, culture, language style, and musical or thematic elements.

Choose only one of the following labels:
- European
- Asian
- African
- Arabic
- North American
- Latino / South American
- Oceania

Return the result in the following format:
LYRICS: <lyrics>  
ETHNICITY: <label>  
KEYWORDS: <list of specific words or expressions from the lyrics that led to your decision>

EXAMPLES:

LYRICS: From Lagos to the stage, I carry my father's name / Afrobeat in my soul, fire in my veins  
ETHNICITY: African  
KEYWORDS: "Lagos", "Afrobeat", "father's name"

LYRICS: I grew up on ramen and neon lights / Tokyo dreams in sleepless nights  
ETHNICITY: Asian  
KEYWORDS: "ramen", "Tokyo", "neon lights"

LYRICS: Raised on southern blues and faded jeans / Mama's voice and Georgia dreams  
ETHNICITY: North American  
KEYWORDS: "southern blues", "faded jeans", "Georgia"

LYRICS: {lyrics}  
ETHNICITY:"""

import re

def getEthnicityAndKeywords(result):
  result = result[promptSize:]
  match = re.search(
    r"ETHNICITY:\s*(European|Asian|African|Arabic|North American|Latino / South American|Oceania)\s*KEYWORDS:\s*([^\n]+)",
    result,
    re.IGNORECASE
  )
  if match:
    ethnie = match.group(1)
    keywords_str = match.group(2).strip()
    # Remove quotes and split by comma
    keywords_list = [kw.strip().replace('"', '').replace('\'', '') for kw in keywords_str.split(',') if kw.strip().replace('"', '').replace('\'', '')]
    keywords_list = list(set(keywords_list))  # Remove duplicates
    return ethnie, keywords_list
  else:
    print("Impossible d'extraire l'ethnie ou les mots-clés.")
    print(result)
    return None, None

def getEthnicityAndReasonLLM(lyrics):
  prompt = prompt_ethnicity(lyrics)
  result = pipe(prompt, max_new_tokens=100, do_sample=False) # Removed temperature=0.7
  output_text = result[0]["generated_text"]
  return getEthnicityAndKeywords(output_text)

In [ ]:
df = pd.ExcelFile(PATH_output_gender)
df = df.parse("Sheet1")
print(getEthnicityAndReasonLLM(df['english_lyrics'][0]))
#if df don't already have 'ethnicity_LLM' key
if 'ethnicity_LLM' not in df.columns or 'keywords_LLM_Ethicity' not in df.columns:
  df['ethnicity_LLM'] = None
  df['keywords_LLM_Ethicity'] = None
  df.to_excel(PATH_output_gender)

In [ ]:
df['ethnicity_LLM'] = df['ethnicity_LLM'].astype('object')
df['keywords_LLM_Ethicity'] = df['keywords_LLM_Ethicity'].astype('object')

for index, row in df.iterrows():
  """if index > 200:
    print("Temps limite atteint.")
    break"""
  if index % 100 == 0:
    clear_output(wait=True)
    print(f"Processing index: {index} / {len(df)}")
  if len(str(df['ethnicity_LLM'][index])) > 3:
    print(df['ethnicity_LLM'][index])
    print(index, end=' ')
    continue
  genre, keywords = getEthnicityAndReasonLLM(df['english_lyrics'][index])
  df.loc[index, 'ethnicity_LLM'] = genre
  df.loc[index, 'keywords_LLM_Ethicity'] = ", ".join(keywords) if keywords else None
  df.to_excel(PATH_output_gender)
  print(index, genre, keywords)

In [ ]:
df = pd.read_excel(PATH_output_gender)
#on affiche le nom des artistes qui ont pour genre "person"
print(df[df['genre'] == 'Person']['track_artist'].unique())
#SAYMYNAME => female
#Aleman => male
#Fili => group
#MiMS => male

df.loc[df['track_artist'] == 'SAYMYNAME', 'genre'] = 'female'
df.loc[df['track_artist'] == 'Aleman', 'genre'] = 'male'
df.loc[df['track_artist'] == 'Fili', 'genre'] = 'group'
df.loc[df['track_artist'] == 'MiMS', 'genre'] = 'male'
df = df.dropna(subset=['genre'])
df.to_excel(PATH_output_gender, index=False)